<a href="https://colab.research.google.com/github/YianKim/Complaints-NLP-Visualization-2021/blob/main/4_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D(dldks).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
from tqdm import tqdm_notebook #process bar
import string #특수문자를 클리닝에서 제거해주고 싶을 때 사용하려고 특수문자를 불러온다. 
import warnings #warning 제거

from gensim import corpora
from gensim import models
import numpy as np
import re
import gzip
import pickle
import matplotlib.pyplot as  plt
from wordcloud import WordCloud
%matplotlib inline
warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
# 제목하고 내용이 list로 받아졌을 때, 중복되지 않는 원소만 편입
def merge제목내용(제목, 내용):
  temp = []
  temp += list(내용)
  for i in 제목:
    if i not in 내용:
      temp.append(i)
  return(temp)

merge제목내용(["A","B","DDD"],["A","CD"]) #['A', 'CD', 'B', 'DDD']

['A', 'CD', 'B', 'DDD']

In [53]:
# 램 한계를 극복하기 위한 글자수 제한
def 글자제한(textvec, LB, UB):
  temp=[]
  for i in textvec:
    if len(i)>LB-1 and len(i)<UB+1:
      temp.append(i)
  return(temp)

글자제한(["A","AA","AAA","AAAA","AAAAA","AAAAAA","AAAAAAA"],3,4)

['AAA', 'AAAA']

In [54]:
# load and uncompress.
with gzip.open('/content/drive/Shareddrives/2021민원/데이터/clean_data_twitter+correction.pickle','rb') as f:
    data = pickle.load(f)

In [55]:
data_overtime=data.drop(["제목", "질문내용", "답변내용", "최상위기관명", "분야명", "담당부서명", "시간"], axis=1)
data_overtime.head()

,제목_nouns,질문내용_nouns,답변내용_nouns,연,월,일,오전오후
0,"[의성군, 귀농, 귀촌, 정책, 문의]","[의성군, 귀농, 귀촌, 정책, 문의]","[의성군, 귀농, 귀촌, 감사, 귀농, 귀촌, 지원, 정책, 설명, 앞서, 귀농, ...",2021,08,27,오후
1,"[불법, 주정]","[불법, 주정, 과태료, 부과, 대상]","[귀하, 가정, 안함, 행복, 기원, 귀하, 국민신문고, 요청, 사항, 문화, 조성...",2021,08,27,오후
2,"[아동, 수당, 지급, 대상]","[아동, 수당, 지급, 대상]","[안녕하십니까, 귀하, 문의, 질의, 대한, 답변, 귀하, 질의, 내용, 아동, 수...",2021,08,27,오후
3,"[광견병, 예방, 접종, 문의]","[광견병, 예방접종, 질문, 사항, 의성군, 광견병, 접종, 예방, 계획, 신지, ...","[안녕하십니까, 귀하, 국민신문고, 통해, 신청, 민원, 대한, 검토, 귀하, 민원...",2021,08,27,오후
4,"[아동, 수당, 지급, 금액, 지급, 일자]","[아동, 수당, 지급, 금액, 지급, 일자]","[안녕하십니까, 귀하, 문의, 질의, 대한, 답변, 귀하, 질의, 내용, 아동, 수...",2021,08,27,오후


# gensim을 이용한 토픽모델링

In [56]:
# 그냥 다 합쳐보기
token_list=[]
for i in range(57249):
  # print(i)
  temp = merge제목내용(data_overtime['제목_nouns'][i], data_overtime['질문내용_nouns'][i])
  token_list.append(merge제목내용(temp, data_overtime['답변내용_nouns'][i]))

In [57]:
dictionary = corpora.Dictionary(token_list)
# dictionary.token2id

In [58]:
term_matrix = [dictionary.doc2bow(token) for token in token_list]
# term_matrix

In [59]:
tfidf = models.TfidfModel(term_matrix)

# for doc in tfidf[term_matrix]:
#     print("doc:")
#     for k, v in doc:
#         print(k, v)

In [60]:
allword=[]
for i in range(len(dictionary.token2id)):
  allword.append((i,1))

In [61]:
weight = { name:value for name, value in zip(list(dictionary.token2id.keys()), [x[1] for x in tfidf[allword]]) }

In [62]:
# fp='/content/drive/MyDrive/서울남산체/08SeoulNamsanB_0.ttf'
# wordcloud = WordCloud(background_color="white", max_words=100, width=4000, height=3000, font_path=fp).generate_from_frequencies(weight)
# plt.figure()
# plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis("off")
# plt.show()

In [69]:
# 토픽모델링

model = models.LdaModel(corpus=term_matrix, id2word=dictionary, num_topics=10)
model.print_topics()

[(0,
  '0.034*"광고" + 0.023*"선박" + 0.021*"위반" + 0.018*"스팸" + 0.016*"표시" + 0.015*"불법" + 0.013*"조치" + 0.011*"행위" + 0.010*"항만" + 0.010*"차량"'),
 (1,
  '0.015*"기관" + 0.013*"시험" + 0.011*"수상기" + 0.011*"교육" + 0.011*"훈련" + 0.009*"해양수산부" + 0.008*"전송" + 0.007*"가능" + 0.007*"검사" + 0.007*"정보통신망법"'),
 (2,
  '0.037*"지원" + 0.013*"신청" + 0.011*"대상" + 0.009*"장애인" + 0.008*"가능" + 0.008*"센터" + 0.007*"지급" + 0.007*"요금" + 0.007*"대상자" + 0.006*"상담"'),
 (3,
  '0.030*"시설" + 0.024*"제조" + 0.022*"관리" + 0.017*"사용" + 0.011*"식품" + 0.010*"안전" + 0.010*"설치" + 0.010*"시행" + 0.009*"공사" + 0.009*"제품"'),
 (4,
  '0.044*"답변" + 0.036*"감사" + 0.035*"처리" + 0.031*"비식" + 0.030*"민원" + 0.030*"숫자" + 0.028*"귀하" + 0.025*"내용" + 0.025*"대한" + 0.024*"사항"'),
 (5,
  '0.030*"제조" + 0.021*"위원회" + 0.017*"규정" + 0.015*"사업자" + 0.013*"허가" + 0.013*"시행" + 0.012*"위원장" + 0.011*"법률" + 0.011*"사업" + 0.009*"사항"'),
 (6,
  '0.026*"정보" + 0.021*"신청" + 0.019*"확인" + 0.017*"인터넷" + 0.015*"가능" + 0.015*"서비스" + 0.012*"홈페이지" + 0.011*"국내" + 0.011*"개인정보" + 0.011*"수신"'),
 (7,
  '

In [70]:
data.loc[3456]

제목                                                 고속도로 갓길주행 전남
제목_nouns                                     [고속도로, 갓길, 주행, 전남]
질문내용                                            고속도로 갓길 주행하여 추월
질문내용_nouns                                   [고속도로, 갓길, 주행, 추월]
답변내용          민원인께서 문의하신 민원내용에 대하여 답변 드립니다 귀하의 민원취지는 고속도로 갓길...
답변내용_nouns    [원인, 문의, 민원, 내용, 답변, 귀하, 민원, 취지, 고속도로, 갓길, 통행,...
최상위기관명                                                      경찰청
분야명                                                    행정/자치/안전
담당부서명                                                       경무과
연                                                          2021
월                                                            08
일                                                            25
오전오후                                                         오후
시간                                                      5:01:41
Name: 3456, dtype: object

In [71]:
model.get_document_topics(term_matrix[3456])

[(0, 0.4976059), (4, 0.24920905), (6, 0.20627144), (7, 0.038088292)]

In [ ]:
spr_ind = np.where(data_overtime["월"]=="03")[0].tolist()+np.where(data_overtime["월"]=="04")[0].tolist()+np.where(data_overtime["월"]=="05")[0].tolist()
sum_ind = np.where(data_overtime["월"]=="06")[0].tolist()+np.where(data_overtime["월"]=="07")[0].tolist()+np.where(data_overtime["월"]=="08")[0].tolist()
aut_ind = np.where(data_overtime["월"]=="09")[0].tolist()+np.where(data_overtime["월"]=="10")[0].tolist()+np.where(data_overtime["월"]=="11")[0].tolist()
win_ind = np.where(data_overtime["월"]=="12")[0].tolist()+np.where(data_overtime["월"]=="01")[0].tolist()+np.where(data_overtime["월"]=="02")[0].tolist()

In [ ]:
token_list.spr_ind

AttributeError: ignored